In [ ]:
%matplotlib inline

#Use in IBM Q Lab

from qiskit import QuantumCircuit, execute, Aer, IBMQ, QuantumRegister, ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *


provider = IBMQ.load_account()
simulator=Aer.get_backend('qasm_simulator')

qi=QuantumRegister(4)
qs=QuantumRegister(2)
qb=QuantumRegister(2)
qe=QuantumRegister(4)
qxs=QuantumRegister(2)
qo=QuantumRegister(1)
c=ClassicalRegister(1)

circuit=QuantumCircuit(qi,qs,qb,qe,qxs,qo,c)

def fun_or(qc,q0,q1,q2):
    qc.x(q0)
    qc.x(q1)
    qc.ccx(q0,q1,q2)
    qc.x(q2)
    qc.x(q1)
    qc.x(q0)

def or3(qc,q0,q1,q2,b,q3):
    fun_or(qc,q0,q1,b)
    fun_or(qc,b,q2,q3)
    qc.reset(b)

def fun_and(qc,q0,q1,q2):
    qc.ccx(q0,q1,q2)

def and3(qc,q0,q1,q2,b,q3):
    fun_and(qc,q0,q1,b)
    fun_and(qc,b,q2,q3)
    qc.reset(b)

def or4(qc,q0,q1,q2,q3,b1,b2,q4):
    or3(qc,q0,q1,q2,b1,b2)
    fun_or(qc,b2,q3,q4)
    qc.reset(b1)
    qc.reset(b2)

#Input
circuit.x(qi[0])
circuit.x(qi[2])
circuit.x(qi[3])

#Control signal
circuit.x(qs[1])

for i in range(0,2):
    circuit.cx(qs[i],qxs[i])
    circuit.x(qxs[i])

and3(circuit,qxs[0],qxs[1],qi[0],qb[0],qe[0])
and3(circuit,qs[0],qxs[1],qi[1],qb[0],qe[1])
and3(circuit,qxs[0],qs[1],qi[2],qb[0],qe[2])
and3(circuit,qs[0],qs[1],qi[3],qb[0],qe[3])
or4(circuit,qe[0],qe[1],qe[2],qe[3],qb[0],qb[1],qo[0])

circuit.measure(qo[0],c[0])

job=execute(circuit,simulator,shots=100)
result=job.result()
counts=result.get_counts(circuit)

print(counts)
#circuit.draw()
#plot_histogram(counts)